# Iterators

In [ ]:
# A sentence as a sequence of words
import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence:
    
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text) # returns list of nonoverlapping matches
        # of the regexs as a list of strings

    def __getitem__(self, index):
        return self.words[index]

    def __len__(self): # using len and getitem, we complete sequence protocol
        return len(self.words)
    
    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text) # generate abbreviated repr

## Iterables
Any object from which the `iter` built in function can obtain an iterator. Objects implementing the __iter__ method returning an iterator are iterable. Sequences are always iterable, as are objets implementing a __getitem__ method that takes 0-based indexes.

In [6]:
s = 'ABC'
for char in s:
    print (char)

A
B
C


In [7]:
s = 'ABC'
it = iter(s)
while True:
    try:
        print(next(it))
    except StopIteration:
        del it
        break

A
B
C


In [8]:
# The abc.Iterator class
class Iterator(Iterable):
    
    __slots__ = ()
    
    @abstractmethod
    def __next__(self):
        'Return the next item from the iterator. When exhausted, raise StopIteration'
        raise StopIteration
    
    def __iter__(self):
        return self
    
    @classmethod
    def __subclasshook__(cls, C):
        if cls is Iterator:
            if (any('__next__' in B.__dict__ for B in C.__mro__) and 
                any('__iter__' in B.__dict__ for B in C.__mor__)):
                return True
        return NotImplemented
    
    

NameError: name 'Iterable' is not defined

In [ ]:
# to check if something is a interator
isinstance(x, abc.Iterator)

In [11]:
# defn:
# Iterator is any object that implements the __next__ no-argument method
# and returns the next item in a series, or raises StopIteration when
# there are no more items. Python iterators also implement the __iter__ method
# so they are *iterable*, as well.

In [12]:
import re

RE_WORD = re.compile('\w+')

class Sentence:
    
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)
    
    def __iter__self(self):
        return SentenceIterator(self.words)

class SentenceIterator:
    
    def __init__(self, words):
        self.words = words
        self.index = 0
        
    def __next__(self):
        try:
            word = self.words[self.index]
        except IndexError:
            raise StopIteration()
        self.index += 1
        return word
    
    def __iter__(self):
        return self
    # note: not necessary here, but it's the right thing to do,
    # in order to pass the issubclass(SentenceIterator, abc.Iterator) test
    # if we had subclasses SentenceIterator from abc.Iterator, we'd inherit
    # the concrete abc.Iterator.__iter__ method

However, this requires management of the internal state of the iterator.

There are better ways.

In [14]:
# Further clarification:
#   Iterables have an __iter__ method that instantiates a NEW iterator every time
#   Iteratoors implement a __next__ method that returns individual items,
#      and an __iter__ method that returns self
# Therefore:
#   iterators are also iterable;  but iterables, are not iterators
#
# An iterable should NEVER act as a iterator over itself.
# - Iterables must implement __iter__ but not next
# - An iterator should always be iterable; __iter__ should just return self


## Generator Functions for more Pythonic iteration

In [15]:
import re

RE_WORD = re.compile('\w+')

class Sentence:
    
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)
        
    def __iter__(self):
        for word in self.words:
            yield word
        return
    
# done!

# Generator Expressions


In [16]:
class ArithmeticProgression:
    
    def __init__(self, begin, step, end=None):
        self.begin = begin
        self.step = step
        self.end = end # None -> 'infinite' series
        
    def __iter__(self):
        result = type(self.begin + self.step)(self.begin)
        forever = self.end is None
        index = 0
        while forever or result < self.end:
            yield result
            index += 1
            result = self.begin + self.step * index # calculate next result

In [17]:
def aritprog_gen(begin, step, end=None):
    result = type(begin + step)(begin)
    forever = end is None
    index = 0
    while forever or result < end:
        yield result
        index += 1
        result = begin + step * index

In [18]:
aritprog_gen(1, 5, 10)

<generator object aritprog_gen at 0x1090d5e10>

In [20]:
ar = aritprog_gen(1, 5, 10)

In [21]:
next(ar)

1

In [22]:
next(ar)

6

In [26]:
list(aritprog_gen(1, 5, 15))

[1, 6, 11]

### Arithmetic Progression with Itertools

In [27]:
import itertools
gen = itertools.count(1, .5)
next(gen)

1

In [28]:
next(gen)

1.5

In [29]:
next(gen)

2.0

In [30]:
gen = itertools.takewhile(lambda n: n < 3, itertools.count(1, .5))
list(gen)

[1, 1.5, 2.0, 2.5]

In [32]:
import itertools

def aritprog_gen(begin, step, end=None):
    first = type(begin + step)(begin)
    ap_gen = itertools.count(first, step)
    if end is not None:
        ap_gen = itertools.takewhile(lambda n: n < end, ap_gen)
    return ap_gen

# Yield From

In [1]:
def chain(*iterables):
    for it in iterables:
        for i in it:
            yield i

s = 'ABC'
t = tuple(range(3))
list(chain(s, t))

['A', 'B', 'C', 0, 1, 2]

In [ ]:
# same result
def chain(*iterables):
    for i interables:
        yield from i

# More functionality from __iter__


In [11]:
# pass a second arg a sentinel value

In [5]:
from random import randint
def d6():
    return randint(1, 6)

d6_iter = iter(d6,1)

In [6]:
d6_iter

<callable-iterator at 0x1024ab2d0>

In [7]:
for roll in d6_iter:
    print roll

2
5


In [10]:
d6_iter = iter(d6,1)
for roll in d6_iter:
    print roll

5
3
5
6
2
4
6
4
5
4
5
4
